# Lab 8: Regex Review, Data Management, and Dates

In [5]:
options(repr.plot.width=6, repr.plot.height=4)

require(tidyverse)
require(stringr)
require(lubridate)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.5
✔ tidyr   0.8.1     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘tibble’ was built under R version 3.4.3”Warning message:
“package ‘tidyr’ was built under R version 3.4.4”Warning message:
“package ‘purrr’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”Warning message:
“package ‘forcats’ was built under R version 3.4.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: lubridate
Warning message:
“package ‘lubridate’ was built under R version 3.4.4”
Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



## Regular Expressions Review

Regular expressions (regex) are a way to describe patterns in text and are used to search for and match certain patterns in strings.

`Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems.` - Jamie Zawinski

For instance, say that you want to find and extract all the email addresses in a document automatically. How might we do that?

### Special characters

Regex takes advantage of several reserved characters that are used for special functions. 

`. \ | ( ) [ ] ^ $ { } * + ?`

### Character classes

* `.` matches anything (wildcard)
* `[aeiou]` matches a single character in the set provided
* `[^aeiou]` matches a single character NOT in the set
* `[a-e]` matches a range, equivalent to `[abcde]`

#### Shorthand

* `\w` matches a "word" character, equivalent to `[a-zA-Z0-9_]`
* `\s` matches any whitespace, including tabs and newlines
* `\d` matches digits, equivalent to `[0-9]`
* `\W`, `\S`, and `\D` match the opposite of the lower-case versions

#### Special characters

* Note that `\t` and `\n` match the tab and newline characters. 
* If you want the "literal" versions of any of the reserved characters, you will need to escape them with a backslash `\`, e.g. `[\.\\\|]`


### Grouping

* `()` are used to group patterns together. This can be used with any of the below operators. This can also be used to extract portions of a regex out individually, which we will later learn.
* `\1`, `\2`, etc. refers to the first, second, etc. group in the match.

### Operators

* `|` is the OR operator and allows matches of either side
* `{}` describes how many times the preceeding character of group must occur:
  * `{m}` must occur exactly `m` times
  * `{m,n}` must occur between `m` and `n` times, inclusive
  * `{m,}` Must occur at least `m` times
* `*` means the preceeding character can appear zero or more times, equivalent to `{0,}`
* `+` means the preceeding character must appear one or more times, equivalent to `{1,}`
* `?` means the preceeding character can appear zero or one time, equivalent to `{0,1}`

### Anchors

* `^` matches the start of a string (or line)
* `$` matches the end of a string (or line)
* `\b` matches a word "boundary"
* `\B` matches not word boundary

### String Functions

See `https://stringr.tidyverse.org/reference/index.html` for a more complete list of string functions and their documentation.

Recall that any functions that use the argument `pattern` in the documentation will by default assume the pattern provided is a regular expression. These include functions like `str_detect`, `str_replace`, `str_count`, etc.


In [129]:
ne_states = c('Connecticut', 'Maine', 'Massachusetts', 'Vermont', 'New Hampshire', 'Rhode Island')

str_length(ne_states)

[1] 11  5 13  7 13 12

In [130]:
str_c('Seoul', 'Korea', sep=', ')
# paste('Seoul', 'Korea', sep=', ')

[1] "Seoul, Korea"

In [132]:
x = c('abc', '123', NA)

str_c('|-', x, '-|')

[1] "|-abc-|" "|-123-|" NA

In [133]:
str_c('|-', str_replace_na(x), '-|')

[1] "|-abc-|" "|-123-|" "|-NA-|"

To collapse a vector of strings, use the `collapse` argument to `str_c`:

In [134]:
str_c(ne_states, collapse=", ")

[1] "Connecticut, Maine, Massachusetts, Vermont, New Hampshire, Rhode Island"

In [17]:
str_replace_all('This is a sentence.', '([aeiouAEIOU])', '\\1\\1\\1')

[1] "Thiiis iiis aaa seeenteeenceee."

In [23]:
str_replace_all('beauty obvious previous quiet serious various', '([aeiou])([aeiou])([aeiou])', '\\3\\2\\1')

[1] "buaety obvuois prevuois qeiut seruois varuois"

### Subsetting Strings

In [136]:
ne_states = c("Connecticut", "Maine", "Massachusetts", "Vermont", "New Hampshire", "Rhode Island")

str_sub(ne_states, 1, 3)

[1] "Con" "Mai" "Mas" "Ver" "New" "Rho"

In [71]:
str_sub(ne_states, -3, -1)

[1] "cut" "ine" "tts" "ont" "ire" "and"

In [137]:
str_sub(ne_states, 1, 7)  # notice that this still works for Maine

[1] "Connect" "Maine"   "Massach" "Vermont" "New Ham" "Rhode I"

In [73]:
str_sub(ne_states, 1, 1) = str_to_lower(str_sub(ne_states, 1, 1))
ne_states

[1] "connecticut"   "maine"         "massachusetts" "vermont"      
[5] "new Hampshire" "rhode Island"

In [74]:
str_sub(ne_states, -3, -1) = str_to_upper(str_sub(ne_states, -3, -1))
ne_states

[1] "connectiCUT"   "maINE"         "massachuseTTS" "vermONT"      
[5] "new HampshIRE" "rhode IslAND"

## Managing Data

The file `reddit_dirty.txt` contains a dirtier version of the reddit comments dataset that you used in Problem Set 6. To see the first few lines, we can use the command:

In [125]:
read_lines('https://raw.githubusercontent.com/rogerfan/stats306_f18_labs/master/reddit_dirty.txt', n_max=10)

[1] "postid: 1"                                                                                                                                                                                                                                                                                                                                                                                                       
 [2] "https://www.reddit.com/user/br_shadow"                                                                                                                                                                                                                                                                                                                                                                           
 [3] "Comment: Thank you for this, there is a person writing to me through facebook that's my only point of communication. He also sent me this, is there maybe a website to check its authenticity ? I was unable to find any lists of registered foundations in Uganda  https://scontent-syd2-1.xx.fbcdn.net/v/t34.0-12/25589975_2043006685934965_29416957_n.jpg?oh=088ca5a5d027b9c7f243b2bbcb0261f4&amp;oe=5A430D87"
 [4] "2017-12-25 15:49:08"                                                                                                                                                                                                                                                                                                                                                                                             
 [5] "postid: 2"                                                                                                                                                                                                                                                                                                                                                                                                       
 [6] "https://www.reddit.com/user/Ksalol"                                                                                                                                                                                                                                                                                                                                                                              
 [7] "Comment: They are not to quick actually. It's mainly the Fentanyl market that's ruining it for the rest of us! But 3f-pvp will probably get banned in 6-10 months, because it's going to be very popular and the goverment hates fun so."                                                                                                                                                                        
 [8] "2017-12-25 17:42:50"                                                                                                                                                                                                                                                                                                                                                                                             
 [9] "postid: 3"                                                                                                                                                                                                                                                                                                                                                                                                       
[10] "https://www.reddit.com/user/itscool83"

### Problem 1
Read in the data from `https://raw.githubusercontent.com/rogerfan/stats306_f18_labs/master/reddit_dirty.txt` (or download the data and read it in locally) and store it in `posts1`. You should get a tibble that looks something like:

```{r}
# A tibble: 40,000 x 1
   var1                                                                        
   <chr>                                                                       
 1 1                                                                           
 2 https://www.reddit.com/user/br_shadow                                       
 3 Comment: Thank you for this, there is a person writing to me through facebo…
 4 2017-12-25 15:49:08                                                         
 5 2                                                                           
 6 https://www.reddit.com/user/Ksalol                                          
 7 Comment: They are not to quick actually. It's mainly the Fentanyl market th…
 8 2017-12-25 17:42:50                                                         
 9 3                                                                           
10 https://www.reddit.com/user/itscool83                                       
# ... with 39,990 more rows
```

In [1]:
posts1 = NA

### Problem 2

Clean up `posts1` so that each row represents a single reddit post and each variable is in its own column. So you should end up with a table that looks like the following. Store this is `posts2`.

```{r}
# A tibble: 10,000 x 4
   postid user                                            body            time 
    <dbl> <chr>                                           <chr>           <chr>
 1      1 https://www.reddit.com/user/br_shadow           Comment: Thank… 2017…
 2      2 https://www.reddit.com/user/Ksalol              Comment: They … 2017…
 3      3 https://www.reddit.com/user/itscool83           Comment: tell … 2017…
 4      4 https://www.reddit.com/user/Glu7enFree          "Comment: Auti… 2017…
 5      5 https://www.reddit.com/user/Theotheogreato      "Comment: You … 2017…
 6      6 https://www.reddit.com/user/Shadrac121          Comment: Hopfu… 2017…
 7      7 https://www.reddit.com/user/1fzUjhemoSB1QV7zI7  Comment: Si ce… 2017…
 8      8 https://www.reddit.com/user/MinisterOfEducation Comment: I don… 2017…
 9      9 https://www.reddit.com/user/AabidS10            Comment: i don… 2017…
10     10 https://www.reddit.com/user/S3RG10              "Comment: I'm … 2017…
# ... with 9,990 more rows
```

In [2]:
posts2 = NA

### Problem 3
Note that for this and the following questions, you may want to have the `lubridate` documentation handy (https://lubridate.tidyverse.org/reference/index.html). 

Further process the data by cleaning up the `user` and `body` fields to remove the url and `"Comment: "` portions. Also note that in `posts2` above the `time` variable is a string. Convert this to a datetime object. Store this cleaned data in `posts3`. It should look like:

```{r}
# A tibble: 10,000 x 4
   postid user                body                          time               
    <dbl> <chr>               <chr>                         <dttm>             
 1      1 br_shadow           Thank you for this, there is… 2017-12-25 15:49:08
 2      2 Ksalol              They are not to quick actual… 2017-12-25 17:42:50
 3      3 itscool83           tell her you guys should han… 2017-12-25 18:54:13
 4      4 Glu7enFree          "Autism is a high honor in t… 2017-12-25 07:48:17
 5      5 Theotheogreato      "You thought a cat was your … 2017-12-25 20:58:08
 6      6 Shadrac121          Hopfully she takes wat peopl… 2017-12-25 22:27:31
 7      7 1fzUjhemoSB1QV7zI7  Si ce propui sa facem cu toa… 2017-12-25 07:41:31
 8      8 MinisterOfEducation I don't mean to be impolite,… 2017-12-25 19:28:35
 9      9 AabidS10            i dont have a 720p x265 of i… 2017-12-25 13:20:32
10     10 S3RG10              "I'm dying to try Guatemalan… 2017-12-25 00:48:46
# ... with 9,990 more rows
```

In [3]:
posts3 = NA

### Problem 4
Sort the data by time of the post. For each post, calculate the duration between that post and the previous post. Write code that plots a histogram of these values, similar to the plot below:

![plot](stats306_lab8_prob4.png)

In [8]:
posts4 = NA

# ggplot(posts4)

### Problem 5
Note that the times are currently in the UTC timezone. Convert these to EST (`"America/New_York"`). Then group the data by hour and calculate the median post length for each hour. Plot this as a line graph:

![plot](stats306_lab8_prob5.png)

In [9]:
posts5 = posts4

# ggplot(posts5)

### Problem 6
Can you make the same plot as Problem 5, but instead grouping posts into 20-minute intervals? This should result in a plot similar to the following:

![plot](stats306_lab8_prob6.png)

In [11]:
posts6 = posts4

# ggplot(posts6)

## Regex Exercises

Given the corpus of common words in `stringr::words`, create regular expressions that find all words that:

- Start with `y` (I've done this one for you)
- End with `ed`, but not with `eed`
- End with the same two-letter sequence they start with (e.g. `church`)


In [12]:
words = stringr::words

In [118]:
str_view(words, "^y\\w*", match=TRUE)

HTML widgets cannot be represented in plain text (need html)

Try to match the valid `dates` below (first row) without matching the invalid dates (the rest).

Hint: Start by writing a pattern that matches all the entries. Then try to refine your pattern to omit the invalid dates.

In [14]:
dates = c('2012-05-13', '2014-12-31', '1991-06-14', '1991/06/14',
          '200a-05-13',  # invalid year
          '2014-15-20',  # invalid month
          '2014-00-20',  # invalid month
          '2016-04-35',  # invalid day
          '2014-12-00',  # invalid day
          '2013/03-25')  # non-matching separators

str_view(dates, '2012')

HTML widgets cannot be represented in plain text (need html)